In [1]:
import numpy as np
import pandas as pd
import time
from sklearn import model_selection, preprocessing, metrics, ensemble
import lightgbm as lgb

In [2]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test_BDIfz5B.csv")
train_df["train_set"] = 1
test_df["train_set"] = 0
print "Train and test shape : ", train_df.shape, test_df.shape
train_df.head()

Train and test shape :  (1023191, 7) (773858, 5)


,id,user_id,campaign_id,send_date,is_open,is_click,train_set
0,42_14051,14051,42,01-09-2017 19:55,0,0,1
1,52_134438,134438,52,02-11-2017 12:53,0,0,1
2,33_181789,181789,33,24-07-2017 15:15,0,0,1
3,44_231448,231448,44,05-09-2017 11:36,0,0,1
4,29_185580,185580,29,01-07-2017 18:01,0,0,1


In [3]:
train_df["send_date"] = pd.to_datetime(train_df["send_date"], format="%d-%m-%Y %H:%M")
test_df["send_date"] = pd.to_datetime(test_df["send_date"], format="%d-%m-%Y %H:%M")
train_df.head()

,id,user_id,campaign_id,send_date,is_open,is_click,train_set
0,42_14051,14051,42,2017-09-01 19:55:00,0,0,1
1,52_134438,134438,52,2017-11-02 12:53:00,0,0,1
2,33_181789,181789,33,2017-07-24 15:15:00,0,0,1
3,44_231448,231448,44,2017-09-05 11:36:00,0,0,1
4,29_185580,185580,29,2017-07-01 18:01:00,0,0,1


In [4]:
train_df["ordinal_date"] = train_df["send_date"].apply(lambda x: time.mktime(x.timetuple()))
test_df["ordinal_date"] = test_df["send_date"].apply(lambda x: time.mktime(x.timetuple()))
train_df.head()

,id,user_id,campaign_id,send_date,is_open,is_click,train_set,ordinal_date
0,42_14051,14051,42,2017-09-01 19:55:00,0,0,1,1.504276e+09
1,52_134438,134438,52,2017-11-02 12:53:00,0,0,1,1.509607e+09
2,33_181789,181789,33,2017-07-24 15:15:00,0,0,1,1.500890e+09
3,44_231448,231448,44,2017-09-05 11:36:00,0,0,1,1.504592e+09
4,29_185580,185580,29,2017-07-01 18:01:00,0,0,1,1.498912e+09


In [5]:
train_df = train_df.sort_values(by="ordinal_date").reset_index(drop=True)
test_df = test_df.sort_values(by="ordinal_date").reset_index(drop=True)
train_df.head()

,id,user_id,campaign_id,send_date,is_open,is_click,train_set,ordinal_date
0,29_170634,170634,29,2017-07-01 18:01:00,0,0,1,1.498912e+09
1,29_7740,7740,29,2017-07-01 18:01:00,0,0,1,1.498912e+09
2,29_20155,20155,29,2017-07-01 18:01:00,0,0,1,1.498912e+09
3,29_134093,134093,29,2017-07-01 18:01:00,0,0,1,1.498912e+09
4,29_82171,82171,29,2017-07-01 18:01:00,0,0,1,1.498912e+09


In [6]:
full_df = train_df.append(test_df).reset_index(drop=True)
full_df.head()

,campaign_id,id,is_click,is_open,ordinal_date,send_date,train_set,user_id
0,29,29_170634,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,170634
1,29,29_7740,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,7740
2,29,29_20155,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,20155
3,29,29_134093,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,134093
4,29,29_82171,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,82171


In [7]:
full_df["user_cum_count"] = full_df.groupby("user_id")["id"].cumcount()
gdf = full_df.groupby("user_id")["id"].size().reset_index()
gdf.columns = ["user_id", "user_count"]
full_df = pd.merge(full_df, gdf, on="user_id")
full_df["user_prev_date"] = full_df.groupby("user_id")["ordinal_date"].shift(1)
full_df["user_prev_camp"] = full_df.groupby("user_id")["campaign_id"].shift(1)
full_df["user_date_diff"] = full_df["ordinal_date"] - full_df["user_prev_date"]
full_df["user_camp_diff"] = full_df["campaign_id"] - full_df["user_prev_camp"]
full_df["hour"] = pd.to_datetime(full_df["send_date"]).dt.hour
#full_df = full_df.drop(["ordinal_date", "send_date", "user_prev_date", "user_prev_camp"], axis=1)

grouped_df = full_df.groupby("user_id")
for name, group in grouped_df:
    print name
    print group
    break

5
         campaign_id    id  is_click  is_open  ordinal_date  \
1201228           32  32_5       0.0      0.0  1.500446e+09   
1201229           53  53_5       0.0      0.0  1.509988e+09   
1201230           55  55_5       NaN      NaN  1.515492e+09   
1201231           60  60_5       NaN      NaN  1.515667e+09   
1201232           66  66_5       NaN      NaN  1.518690e+09   
1201233           70  70_5       NaN      NaN  1.518775e+09   
1201234           73  73_5       NaN      NaN  1.519298e+09   

                  send_date  train_set  user_id  user_cum_count  user_count  \
1201228 2017-07-19 12:02:00          1        5               0           7   
1201229 2017-11-06 22:33:00          1        5               1           7   
1201230 2018-01-09 15:35:00          0        5               2           7   
1201231 2018-01-11 16:04:00          0        5               3           7   
1201232 2018-02-15 15:51:00          0        5               4           7   
1201233 2018-02-16 

In [8]:
"""
full_df = full_df.sort_values(by=["user_id", "ordinal_date"]).reset_index(drop=True)

prev_user = 1
first_open_list = []
second_open_list = []
third_open_list = []
first_click_list = []
second_click_list = []
third_click_list = []
open_list = []
click_list = []
for ind, row in full_df.iterrows():
    if row['user_id'] != prev_user:
        if not np.isnan(row['is_open']):
            open_list = [row['is_open']]
            click_list = [row['is_click']]
        first_open_list.append(-99)
        second_open_list.append(-99)
        third_open_list.append(-99)
        first_click_list.append(-99)
        second_click_list.append(-99)
        third_click_list.append(-99)
        prev_user = row['user_id']
    else:
        if len(open_list) > 0:
            first_open_list.append(open_list[0])
            first_click_list.append(click_list[0])
        else:
            first_open_list.append(-99)
            first_click_list.append(-99)
            
        if len(open_list) > 1:
            second_open_list.append(open_list[1])
            second_click_list.append(click_list[1])
        else:
            second_open_list.append(-99)
            second_click_list.append(-99)
            
        if len(open_list) > 2:
            third_open_list.append(open_list[2])
            third_click_list.append(click_list[2])
        else:
            third_open_list.append(-99)
            third_click_list.append(-99)
            
        if not np.isnan(row['is_open']):
            open_list.append(row['is_open'])
            click_list.append(row['is_click'])
            
full_df["first_open"] = first_open_list
full_df["first_click"] = first_click_list
full_df["second_open"] = second_open_list
full_df["second_click"] = second_click_list
full_df["third_open"] = third_open_list
full_df["third_click"] = third_click_list

print full_df.head()
"""

'\nfull_df = full_df.sort_values(by=["user_id", "ordinal_date"]).reset_index(drop=True)\n\nprev_user = 1\nfirst_open_list = []\nsecond_open_list = []\nthird_open_list = []\nfirst_click_list = []\nsecond_click_list = []\nthird_click_list = []\nopen_list = []\nclick_list = []\nfor ind, row in full_df.iterrows():\n    if row[\'user_id\'] != prev_user:\n        if not np.isnan(row[\'is_open\']):\n            open_list = [row[\'is_open\']]\n            click_list = [row[\'is_click\']]\n        first_open_list.append(-99)\n        second_open_list.append(-99)\n        third_open_list.append(-99)\n        first_click_list.append(-99)\n        second_click_list.append(-99)\n        third_click_list.append(-99)\n        prev_user = row[\'user_id\']\n    else:\n        if len(open_list) > 0:\n            first_open_list.append(open_list[0])\n            first_click_list.append(click_list[0])\n        else:\n            first_open_list.append(-99)\n            first_click_list.append(-99)\n      

In [9]:
gdf = full_df.groupby("user_id")["ordinal_date"].agg(["min", "mean", "max", "std"]).reset_index()
gdf.columns = ["user_id", "user_min_date", "user_mean_date", "user_max_date", "user_std_date"]
full_df = pd.merge(full_df, gdf, on="user_id")
full_df.head()

,campaign_id,id,is_click,is_open,ordinal_date,send_date,train_set,user_id,user_cum_count,user_count,user_prev_date,user_prev_camp,user_date_diff,user_camp_diff,hour,user_min_date,user_mean_date,user_max_date,user_std_date
0,29,29_170634,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,170634,0,13,NaN,NaN,NaN,NaN,18,1.498912e+09,1.510627e+09,1.520254e+09,8.289203e+06
1,30,30_170634,0.0,0.0,1.499243e+09,2017-07-05 14:00:00,1,170634,1,13,1.498912e+09,29.0,331140.0,1.0,14,1.498912e+09,1.510627e+09,1.520254e+09,8.289203e+06
2,36,36_170634,0.0,0.0,1.501761e+09,2017-08-03 17:13:00,1,170634,2,13,1.499243e+09,30.0,2517180.0,6.0,17,1.498912e+09,1.510627e+09,1.520254e+09,8.289203e+06
3,42,42_170634,0.0,0.0,1.504276e+09,2017-09-01 19:49:00,1,170634,3,13,1.501761e+09,36.0,2514960.0,6.0,19,1.498912e+09,1.510627e+09,1.520254e+09,8.289203e+06
4,44,44_170634,0.0,0.0,1.504591e+09,2017-09-05 11:33:00,1,170634,4,13,1.504276e+09,42.0,315840.0,2.0,11,1.498912e+09,1.510627e+09,1.520254e+09,8.289203e+06


In [10]:
pivot_df = pd.pivot_table(full_df, values="train_set", index="user_id", columns="campaign_id", aggfunc="count", fill_value=0).reset_index()
#print pivot_df.columns
#pivot_df = pivot_df.drop(["campaign_id"], axis=1)
pivot_df.columns = ["user_id"] + ["camp_"+str(i) for i in range(29,81)]
full_df = pd.merge(full_df, pivot_df, on="user_id")
full_df.head()

,campaign_id,id,is_click,is_open,ordinal_date,send_date,train_set,user_id,user_cum_count,user_count,...,camp_71,camp_72,camp_73,camp_74,camp_75,camp_76,camp_77,camp_78,camp_79,camp_80
0,29,29_170634,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,170634,0,13,...,1,0,0,1,1,0,0,0,0,0
1,30,30_170634,0.0,0.0,1.499243e+09,2017-07-05 14:00:00,1,170634,1,13,...,1,0,0,1,1,0,0,0,0,0
2,36,36_170634,0.0,0.0,1.501761e+09,2017-08-03 17:13:00,1,170634,2,13,...,1,0,0,1,1,0,0,0,0,0
3,42,42_170634,0.0,0.0,1.504276e+09,2017-09-01 19:49:00,1,170634,3,13,...,1,0,0,1,1,0,0,0,0,0
4,44,44_170634,0.0,0.0,1.504591e+09,2017-09-05 11:33:00,1,170634,4,13,...,1,0,0,1,1,0,0,0,0,0


In [11]:
gdf = full_df[full_df['is_click']==1]
gdf = gdf.groupby("user_id")["ordinal_date"].agg(["mean","std"]).reset_index()
gdf.columns = ["user_id", "user_mean_date_click", "user_std_date_click"]
full_df = pd.merge(full_df, gdf, on="user_id", how="left")
full_df.head()

,campaign_id,id,is_click,is_open,ordinal_date,send_date,train_set,user_id,user_cum_count,user_count,...,camp_73,camp_74,camp_75,camp_76,camp_77,camp_78,camp_79,camp_80,user_mean_date_click,user_std_date_click
0,29,29_170634,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,170634,0,13,...,0,1,1,0,0,0,0,0,NaN,NaN
1,30,30_170634,0.0,0.0,1.499243e+09,2017-07-05 14:00:00,1,170634,1,13,...,0,1,1,0,0,0,0,0,NaN,NaN
2,36,36_170634,0.0,0.0,1.501761e+09,2017-08-03 17:13:00,1,170634,2,13,...,0,1,1,0,0,0,0,0,NaN,NaN
3,42,42_170634,0.0,0.0,1.504276e+09,2017-09-01 19:49:00,1,170634,3,13,...,0,1,1,0,0,0,0,0,NaN,NaN
4,44,44_170634,0.0,0.0,1.504591e+09,2017-09-05 11:33:00,1,170634,4,13,...,0,1,1,0,0,0,0,0,NaN,NaN


In [12]:
gdf = full_df[full_df['is_open']==1]
gdf = gdf.groupby("user_id")["ordinal_date"].agg(["mean","std"]).reset_index()
gdf.columns = ["user_id", "user_mean_date_open", "user_std_date_open"]
full_df = pd.merge(full_df, gdf, on="user_id", how="left")
full_df.head()

,campaign_id,id,is_click,is_open,ordinal_date,send_date,train_set,user_id,user_cum_count,user_count,...,camp_75,camp_76,camp_77,camp_78,camp_79,camp_80,user_mean_date_click,user_std_date_click,user_mean_date_open,user_std_date_open
0,29,29_170634,0.0,0.0,1.498912e+09,2017-07-01 18:01:00,1,170634,0,13,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN
1,30,30_170634,0.0,0.0,1.499243e+09,2017-07-05 14:00:00,1,170634,1,13,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN
2,36,36_170634,0.0,0.0,1.501761e+09,2017-08-03 17:13:00,1,170634,2,13,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN
3,42,42_170634,0.0,0.0,1.504276e+09,2017-09-01 19:49:00,1,170634,3,13,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN
4,44,44_170634,0.0,0.0,1.504591e+09,2017-09-05 11:33:00,1,170634,4,13,...,1,0,0,0,0,0,NaN,NaN,NaN,NaN


In [16]:
full_df["camps_sent"] = (full_df['camp_29'].astype('str') + full_df['camp_30'].astype(str) + 
    full_df['camp_31'].astype('str') + full_df['camp_32'].astype(str) +
    full_df['camp_33'].astype('str') + full_df['camp_34'].astype(str) +
    full_df['camp_35'].astype('str') + full_df['camp_36'].astype(str) +
    full_df['camp_37'].astype('str') + full_df['camp_38'].astype(str) +
    full_df['camp_39'].astype('str') + full_df['camp_40'].astype(str) +
    full_df['camp_41'].astype('str') + full_df['camp_42'].astype(str) +
    full_df['camp_43'].astype('str') + full_df['camp_44'].astype(str) +
    full_df['camp_45'].astype('str') + full_df['camp_46'].astype(str) +
    full_df['camp_47'].astype('str') + full_df['camp_48'].astype(str) +
    full_df['camp_49'].astype('str') + full_df['camp_50'].astype(str) +
    full_df['camp_51'].astype('str') + full_df['camp_52'].astype(str) +
    full_df['camp_53'].astype('str') + full_df['camp_54'].astype(str) +
    full_df['camp_55'].astype('str') + full_df['camp_56'].astype(str) +
    full_df['camp_57'].astype('str') + full_df['camp_58'].astype(str) +
    full_df['camp_59'].astype('str') + full_df['camp_60'].astype(str) +
    full_df['camp_61'].astype('str') + full_df['camp_62'].astype(str) +
    full_df['camp_63'].astype('str') + full_df['camp_64'].astype(str) +
    full_df['camp_65'].astype('str') + full_df['camp_66'].astype(str) +
    full_df['camp_67'].astype('str') + full_df['camp_68'].astype(str) +
    full_df['camp_69'].astype('str') + full_df['camp_70'].astype(str) +
    full_df['camp_71'].astype('str') + full_df['camp_72'].astype(str) +
    full_df['camp_73'].astype('str') + full_df['camp_74'].astype(str) +
    full_df['camp_75'].astype('str') + full_df['camp_76'].astype(str) +
    full_df['camp_77'].astype('str') + full_df['camp_78'].astype(str) +
    full_df['camp_79'].astype('str') + full_df['camp_80'].astype(str)).astype('float')

In [17]:
full_df = full_df.drop(["ordinal_date", "send_date", "user_prev_date", "user_prev_camp"], axis=1)

In [18]:
train_df = full_df[full_df["train_set"] == 1]
test_df = full_df[full_df["train_set"] == 0]
train_df.to_csv("../input/train_feat.csv", index=False)
test_df.to_csv("../input/test_feat.csv", index=False)

In [15]:
np.sort(train_df.campaign_id.unique())

array([29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
       46, 47, 48, 49, 50, 51, 52, 53, 54])

In [16]:
np.sort(test_df.campaign_id.unique())

array([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80])

In [17]:
help(pd.pivot_table)

Help on function pivot_table in module pandas.core.reshape.pivot:

pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')
    Create a spreadsheet-style pivot table as a DataFrame. The levels in
    the pivot table will be stored in MultiIndex objects (hierarchical
    indexes) on the index and columns of the result DataFrame
    
    Parameters
    ----------
    data : DataFrame
    values : column to aggregate, optional
    index : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can contain any of the other types (except list).
        Keys to group by on the pivot table index.  If an array is passed,
        it is being used as the same manner as column values.
    columns : column, Grouper, array, or list of the previous
        If an array is passed, it must be the same length as the data. The
        list can c